# Flight Delay Prediction: Analysis and Implementation

Link to **this** notebook in Databricks: https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/3984215836269142/command/3984215836269143

Link to **EDA** notebook in Databricks:
https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/4444148632306754

Link to **data pipeline** notebook in Databricks: https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/4444148632306347

Link to **Hyperparameter Tuning Random Forest and Gradient Boosted Tree** Notebooks in Databricks: 
Random Forest and LightGBM: https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/3984215836255312

Link to **Hyperparamter Tuning Neural Network**: https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/3984215836270281/command/3984215836270291

Link to **Final Runs: Experiments 1-3 (BEFORE DOWNSAMPLING)** Notebook in Databricks:
https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/3984215836277610/command/3984215836278009

Link to **Final Runs Experiments 4-5 (WITH DOWNSAMPLING)** Notebook in Databricks:
https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/4444148632306707

Link to **Gap Analysis** notebook in Databricks:
https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/4444148632305542

- Team 3-2 
  - Phase 4 Leader - Jess
  - W261 - Summer 2023, Section 5

|| | | |
|-------|-------|-------|-------|
| Kisha Kim <br> kisha.kim@berkeley.edu <br> **Lead in Phase 1** <br> Plan the Project <br> <br> ![Kisha](https://i.imgur.com/LeBnAhBb.jpg) | Chase Madson <br> chase_madson@berkeley.edu <br> **Lead in Phase 2** <br> Pipelines and Baselines <br> <br> ![Chase](https://i.imgur.com/1YjZSlwb.jpg) | Eric Danforth <br> edanforth85@berkeley.edu <br> **Lead in Phase 3** <br> Improve our Baseline <br> <br> ![Eric](https://i.imgur.com/f9pKc1Sb.jpg) | Jess Stockham <br> jhsmith@berkeley.edu <br> **Lead in Phase 4** <br> Find Optimal Algorithm <br> <br> ![Jess](https://i.imgur.com/TNc2GXKb.jpg) |

***https://docs.google.com/spreadsheets/d/1s9vRM4yVHMS86UlCq4orkgHVhLRaXO_E3X_TzdnWbew/edit#gid=0***

![Pipeline](https://i.imgur.com/1wwsXPY.png)

# I. Abstract

This project develops a machine learning model that predicts flight delays based on historical flight, airport station, and weather data spanning five years from 2015-2019 in the United States.  A flight delay is a flight that departs at least 15 minutes past its scheduled time, per the FAA definition. We evaluate the usefulness of our model with a metric that is appropriate for the nature of the data and the business context. Flight delays are relatively rare (about 20 percent of flights) which suggests a focus on either precision or recall. Specifically, we use the f-beta evaluation metric that strikes a balance between recall and precision, but places greater weight on recall to score our various predictive models.  We prioritize recall, which is the number of flights we correctly predicted as delayed divided by all the flights that were truly delayed. This biases us to train a model that errors on the side of predicting more flight delays and being wrong because we do not want to miss any potential delays. It is easier for airlines and customers to recover logistically and reputationally from warning customers there may be a delay and being wrong. This is in contrast to using a more conservative metric that prioritizes precision that only announces a flight delay if we are extremely sure it will happen. A f-beta score ranges from 0 to 100% and an ideal f-beta score is 100%, but a good f-beta score would be between 80% and 100%.

We use data from 2015-2018 to build a model so we can predict flight delays on an unseen dataset of all flights in 2019. This is a big-data problem as the dataset has approximately 31 million rows. We process the data on a distributed cloud computing environment. We built models with 81 features (numeric and categorical) across multiple estimators using data from 2015-2018: logistic regression, random forest, a variant of gradient boosted trees, and neural networks. We also applied innovative approaches such as downsampling the majority class to address the class inbalance and engineering sophisticated graph-based features such as pagerank and triangle count. Our data pipeline is carefully designed to avoid data leakage that would harm the predictive power of the model. Data leakage is a common hazard of machine learning, particularly with time-series data, where researchers use information that would not be available during training to the test set.

In machine learning, you build a model based on data from your training set, in our case from 2015-2018, and then test your model on an unseen test set, in our case 2019 data. Our baseline model was a simple random forest classifier a small set of predictors and default parameters using 3 months of training data from 2015, which had a f-beta score of 0%, which is equivalent to the naive baseline that assumes all flights are on-time. Our final model shows marked improvement from baseline, using a more predictive set of features and downsampling the majority class, predicting the full 2015-2018 training with a f-beta of 63%. We used this final model built on the training set to predict the unseen, held-out 2019 data, which is the best and final assessment of the power of our model, achieving a middling f-beta score of 54%. In our current form, our model's f-beta metric is insufficient to provide a useful tool for industry.  This implies that our features are not rich enough to represent the complex set of factors that cause flight delays. The report's conclusion discusses several directions for future work to improve this predictive tool.

# II. Business Case

On-time Performance (OTP) for air travel has been in a state of steady decline. The Bureau of Transportations statistics show that for the most recent 15+ years of reported data, while weather delays have reduced their overall percentage impact (newer fleets and better technology), air carrier delays have more than made up the difference and the same problems that were evident in the early 2000s worsened during the pandemic and have not improved post-COVID. ([Source](https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations))

The cost to society is not only felt in a direct economic impact to the cost of travel, but in spillover effects to tourism and local economies. Major studies have indicated that this has been caused by the rapid expansion of air carriers. ([Source](https://www-jstor-org.libproxy.berkeley.edu/stable/ee727ffb-b4c6-34ac-9f85-5070b8fe6c05?seq=14)) However, the last decade or so has been marked by mergers - Delta and Northwest (2010), United and Continental (2010), Southwest and AirTran (2011), American Airlines and US Airways (2013), and the recently stalled Frontier and Spirit merger which may result in Spirit and JetBlue joining forces. ([Source](https://www.sheffield.com/2022/the-recent-history-of-u-s-airline-mergers.html), [Source](https://www.forbes.com/sites/marisadellatto/2022/07/27/spirit-and-frontier-airlines-cancel-merger-plans-opening-door-for-jetblues-offer/?sh=4e4cc0723fa1)) Despite all of the streamlining in operations, OTP continues to lag behind.

While the airline industry expects to be profitable for the first time since the start of the pandemic in 2023 ([Source](https://www.cnn.com/2023/06/05/business/iata-airlines-profits-post-pandemic/index.html)), time is money and airlines need to improve this key performance metric if they want to recover from multi-year losses. The most recent estimates by the FAA in 2019 show the cost of airline delays steadily rising from $19.2B in 2012 to an estimated $33B in 2019. ([Source](https://www.faa.gov/air_traffic/by_the_numbers/media/Air_Traffic_by_the_Numbers_2023.pdf)) 

This project develops a predictive tool to predict flight delays. To align with the Federal Aviation Administration's definition, our flight delay outcome metric is a departure time that occurs more than 15 minutes than originally scheduled. We will make predictions 2 hours prior to the flight's scheduled departure time. Importantly, this means we cannot include any data/features in our predictive model that we would ONLY know at the last minute, or more specifically, ONLY know within two hours of the flight's scheduled departure. This is a **classification problem**, where the positive class (y=1) is a flight departure delay and the negative class (y=0) is an on-time flight departure.

We have **imbalanced classes**, where only roughly 20% of flights are delayed (y=1) and 80% are on-time (y=1), and so we tailored our evaluation metric to the nature of the data. Specifically, we evaluated the performance of our model using a **F-beta score, where beta=2**. F-scores balance both precision and recall. Our variant on the F-score, places greater weight on recall and less weight on precision because of the business context.  We focus on recall, which is the number of flights we correctly predicted as delayed divided by all the flights that were truly delayed. This biases us to train a model that errors on the side of predicting more flight delays and being wrong because we do not want to miss any potient delays. This is in contrast to using a more conservative metric focused on precision that only announces a flight delay if we are extremely sure it will happen. We focus on recall because is easier for airlines and customers to recover logistically and reputationally from warning customers there may be a delay and being wrong, than the reverse. F-scores range from 0% to 100%, with 100% as the ideal metric. The most basic baseline is to assume that all flights are on-time (not delayed) and that assumes a recall and f-beta score of 0%.

The f-beta score with beta=2 formula is given by:

$$ F_\beta = \frac{(1 + \beta^2) \cdot \text{Precision} \cdot \text{Recall}}{\beta^2 \cdot \text{Precision} + \text{Recall}} $$

# III. Data and Feature Engineering

In this section, we describe our target variable, the dataset, our exploratory data analysis, feature engineering, and final set of features for the modeling process.

### A. Data

**Target Variable**. The target variable is `DEP_DEL15`, which takes on the value 1 if the flight departed 15 minutes or later than the scheduled departure time, and a 0 otherwise. As discussed above, most flights depart on time, leading to an imbalanced dataset where only about 1 out of 5 flights are delayed.

**Data Size**. The dataset contains all domestic United States flights from the years 2015-2019 and contains approximately 31 million rows. This is a large enough dataset that we consider it "big data," and processed the data in an distributed cloud computing environment (Azure) with 36 cores with 4 worker machines (32 cores) and 1 driver (4 cores). As a basic cleaning step, we removed "CANCELED" flights (1.5% of flights), which never took off, and out of scope for our analysis.

**Data Features**. From this dataset, we have 215 potential flight attributes available to us with which to predict a delayed departure. We describe the variable groups below.
* 63 flight-related features from the [Reporting Carrier On-Time Performance Dataset (BTS)](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ), which contains the following groupings:
  - **Time Period**: The date of the flight is stored in `FL_DATE`, accompanied by 5 derived time attribute features: year, quarter, month, day-of-month, and day-of-week. We selected `DAY_OF_WEEK`, `MONTH`, and `YEAR` as features.
  - **Airline**: There are 5 features about the airline, the aircraft, and the flight number. We select the airline carrier (e.g., Southwest vs Delta) `OP_UNIQUE_CARRIER` as a feature.
  - **Origin & Destination**: There are 20 features describing the flight's points of origin and destination (e.g., airport, city, state). We selected each flight's `ORIGIN` and `DEST` as features. This is a large categorical variable with hundreds of possible values.
  - **Departure & Arrival Performance**: Along with the target variable `DEP_DEL15` itself, there are 17 features describing how the flight departed and landed. Most of these will not be available at prediction-time and therefore are not useful for us. We only used `CRS_DEP_TIME`, which is the scheduled departure time from the Central Reservation System and known at prediction-time, and binned this variable into time groups (e.g., early morning). This is described in the feature engineering section below.
  - **Flight Summaries**: There are 6 features used to summarize how the flight went (e.g., total distance, time in the air, number of flights). We used `DISTANCE` as a feature or the distance between airports in miles, as this information will be known at prediction-time. 
  - **Cause of Delay, Cancelations & Diversions**, **Gate Return Information at Origin Airport**: Information about the cause of delay, cancelations and diversions, and gate return information would not be available to us at prediction time and so are outside our scope. 
* 123 weather-related features brought over from the [Local Climatological Dataset (NOAA)](https://www.ncei.noaa.gov/pub/data/cdo/documentation/LCD_documentation.pdf). We ultimately selected only `ELEVATION` and `AIRPORT_TYPE` (e.g., large, medium, small, seaplane-based) that describe the weather station closest to the airport, as general weather indicators. The hourly and daily weather data across dozens of indicators was too complex for this short analysis. We may develop more advanced weather features in the future.

**Exploratory Data Analysis** Exploratory data analysis helps the researcher understand the trends in each potential feature. We examined the distribution of the data in our features (e.g., histograms, summary statistics) and how categorical features varied with the outcome variable, reviewed levels of missingness, and checked for duplicates. We present several charts to depict how features vary with the outcome variable below on the 2015-2018 dataset. In summary, these trends highlight the importance of time-based features to represent seasonality changes (e.g., time of day, day of week, month of year) and the airline carrier for our modeling. We visualize a few of our findings below:

![Time Series Decomposition](https://i.imgur.com/Fk3DlKd.png)

**Key Insights**
* The trend in delay rates dropped below 17% around 2016 but reverted to a more typical 18% afterwards, showcasing a persistent pattern. 
* Throughout any given year, the delay rate fluctuates about 4% in either direction. Notably, Summer and holiday months often exhibit the highest delays, while Spring and Fall tend to experience below-average delays.
* Even after decomposing the time series to remove trend and seasonality, we still see about a 2% swing in the delay rate in any given month. 

![Delay Rates by Time of Week](https://i.imgur.com/hlQgpGS.png)

**Key Insights**
* Afternoon flights tend to have the most delay.
* Early morning flights see much more variation, likely due to their smaller relative frequency.
* Evening flights on Thursday and Fridays have almost a one-third chance of being delayed, suggesting potential operational challenges during these time frames.

![Delay Rates by Airline](https://i.imgur.com/k8hBK5R.png)

**Key Insights**
* Most smaller airlines fall above the overall delay rate, with JetBlue (B6) and Frontier (F9) being especially prone to delays.
* Larger airlines like Delta (DL) and American Airlines (AA) are more consistent and fall closer to the overall delay rate.
* Hawaii Airlines (HA) and PSA (OH) seem to be better at minimizing delays.

**Missing Data**. We assessed the level of missingness on our features we used in this analysis (or variables we used to engineer new features) in the 2015-2018 dataset. The only variable in this set with missing values was AIR_TIME, which we used to engineer a new feature, which had roughly one-quarter of its values missing. We decided to impute the mean to fill in missing values for numeric features. We used mean rather than median, as median is less computationally burdensome for the big data world. 

**Duplicates**. We checked for duplicates across a composite key (carrier, flight number, origin, flight date, tail number, and scheduled departure time) and found zero duplicates in our dataset.

### B. Feature Engineering

We created several additional features including (1) an indicator for if the flight date is a holiday, (2) an indicator for whether the flight is the first of the day, and (3) binning the scheduled departure dates into chunks (early morning, morning, afternoon, early evening, late at night) to indicate time of day, as well as the graph features and time-based features described below. 

**Graph Features.** Domestic flights between airports can be effectively represented as a network, where airports serve as nodes and the flights between them act as directed links. This alternative way to represent the data allows us to harness the power of graph algorithms to gain valuable insights into the characteristics of airports and their connections.

As an innovative approach, one of the key algorithms we employ is **PageRank**, which helps us assess the importance of an airport within the network based on the number of inbound flights it receives. By understanding the PageRank of an airport, we can identify major hubs and critical transit points in the network. Moreover, an airport's **Triangle Count** assists in revealing the structural properties of the network, highlighting airports with a high number of interconnected flights. Additionally, we attempted to use **Label Propagation Analysis (LPA)** to uncover potential communities or clusters of airports with similar traffic patterns and shed light on regional connectivity and passenger flow dynamics, however this feature proved computationally taxing and the number of labels difficult to reproduce on validation and test data. 

To implement this, we convert our flight data into a graph format and use the algorithms available in the GraphFrames library. Importantly, we mitigate data leakage by computing these metrics solely on the training data, and subsequently, we integrate the values into the test set based on origin and destination airports. Specifically, we implement this at the training sets of each fold during cross-validation. This generated six new features - two sets for each of PageRank, LPA, and Triangle Count. This approach allows us to enhance the understanding of airport characteristics and improve the accuracy of our predictive models for various airport-related analyses. We ultimately dropped LPA from our feature set because it was too compute-intensive.

**Flight Delay Duration (from prior flight that day)**. Flight delays from earlier in the day can have a ripple effect on subsequent flights. This field showcases the average delay in minutes from the same day for the same tail number. We discuss this feature further in the data leakage section of the report.

**Average Flight Delay Duration (from prior day)**. Some flights undergo similar delays (e.g. if there was a repair in one route, it may impact the delay the next day). This field showcases the average delay in minutes from the previous day for the same flight number.

**Average Flight Duration by Flight Number (from prior day)**. Flight duration can help predict flight delay, as long flights might be more prone to delays due to the complexity in coordinating longer routes and schedules. As we are predicting flight delay prior to knowing the current flight duration, we use the average flight duration from the previous day for the same route. 

**Expected Airport Congestion**. The number of flights scheduled at the airport at the departure time block could impact flight delay. Higher number of flights scheduled from the same airport at the same time could lead to flight delays due to flight coordination/traffic complexities. Since scheduled flight information is retrievable prior to 2 hours, there were no lags used to create this new feature.

### 3. Data Dictionary for Features Selected
Below, we show a data dictionary for the features included in our modeling. Notably, ORIGIN and DEST were only part of the baseline model, but dropped when we did further experiments. These two features are enormous categorical variables that represent every airport in the United States. Instead, as we moved forward from the baseline, we engineered several other features that can capture airport characteristics in a more efficient manner, including pagerank, triangle count, and a measure of airport congestion that day. In the table below, we use an \* to indicate features we added after the baseline model. We included all features, with the exception of ORIGIN and DEST, in the hypertuning and final model runs.

**Table 1. Feature Data Dictionary**
|Feature|	Type|	Example Data|Feature Type | Description| 
| ----------- | ----------- | ----------- | ----------- | ----------- |
|MONTH|	Int	|1| Categorical Feature| Month of flight (Jan = 1, ...) | 
|DAY_OF_WEEK|	Int	|4| Categorical Feature| Day of flight (Mon = 1, ...) |
|YEAR|	Int	|4| Categorical Feature| Year of flight (2015 = 1, ...) |
|OP_UNIQUE_CARRIER|	string	|AA| Categorical Feature| Unique AITA Code for Airline | 
|ORIGIN (basline model only)|	String	|SFO| Categorical Feature| AITA Code for the Airport that Flight Departs From | 
|DEST (basline model only)|	String	|JFK| Categorical Feature| AITA Code for the Airport that Flight Arrives In |
|DISTANCE|	Int	|2427| Numerical Feature| Miles Between Origin and Destination Airports |
|origin_type|	string	| medium_airport| Categorical Features| Whether Origin Airport is Big/Med/Small |
|dest_type|	string	|medium_airport| Categorical Features| Whether Destination Airport is Big/Med/Small |
|ELEVATION|	Int	|1462| Numerical Features| Elevation of Station Weather Reported |
|CRS_DEP_BUCKET|	Binned into ranges	|0900-1200| Categorical Features| The Scheduled Departure Time, Bucketized |
|is_holiday\* | Int | 1|  Categorical Feature | If the flight date is a holiday
|is_holiday_adjacent\* | Int | 1|  Categorical Feature | If the flight date is the day before or the day after a holiday
|pagerank_origin\*, pagerank_dest\* | Double| 0.9876 | Numerical Feature | Importance of airport based on inbound flights
|triangle_count_origin\*, triangle_count_dest\* | Double | 3.0 | Numerical Feature | Higlights airports with a large number of interconnected flights
|FE_PRIOR_DAILY_AVG_DEP_DELAY\* | Double |8.5  | Numerical Feature| Average Flight Delay Duration (from prior day) |
|DEP_DELAY_LAG\* | Double | 15.0 | Numerical Feature| For the aircraft slated to make the current trip we seek to predict, this provides the departure delay (in minutes) of its preceding flight |
|FE_PRIOR_AVG_DURATION\* | Double |56.70454 | Numerical Feature| Average Flight Duration by Flight Number (from prior day) |
|FE_NUM_FLIGHT_SCHEDULED\* | Integer | 55 | Numerical Feature | Expected Airport Congestion |

\* Engineered Features

# IV. Data Leakage
Data Leakage is when we use information from outside the training set to build our model. In time-series data, we must be particularly careful to not use data from the future (that we would not have at the time of prediction 2 hours before the flight's scheduled departure) to build our model. Data leakage causes less accurate prediction tools. Our data pipeline, as described below in Section IV is carefully designed to minimize leakage. For example, we apply the meta-data from our training set to the validation set (or the test set, depending on the pipeline stage) to impute missing and normalize numeric data, as well as to create our graph features. In addition, our cross-validation is a time-series blocked design where the folds do not overlap, which restricts any data leakage from the future.

Equally important, when we did feature selection, we were careful to select and create new features with information that we would know more than two hours before the flight. We did keep one feature in the model that may have a minimal amount of leakage: DEP_DELAY_LAG. This attribute captures the departure delay (in minutes) of an aircraft's immediately preceding flight — the one directly preceding the flight we seek to predict. Admittedly, a fraction of short-haul flights may take off within two hours of their subsequent scheduled departure, rendering this information unavailable at prediction-time in some cases. However, we believe this introduces an inconsequential amount of data leakage to our model. In actual practice, we would follow a more sophisticated flowchart to determine whether the aircraft was on-schedule or not to eliminate leakage. However, the advanced version of this feature entails several resource-intensive operations — including verifying the flight's arrival status, confirming the flight's departure from the prior airport, converting from local time to UTC, and more. Given these complexities, we are content with the minimal leakage it introduces.

# V: Modeling Pipeline

### A. Data Processing Steps
We intentionally designed a data pipeline to avoid data leakage. We describe our data pipeline in detail in the image below. In STEP 1, we cleaned the full 2015-2019 dataset. We dropped canceled flights, created new features, and one-hot encoded categorical features. In STEP 2, we then split the dataset into a train (2015-2018) and test dataset (2019). The test dataset is held out until the very last step of the pipeline (step 7).

In STEP 3, we then prepared the data for cross-validation hyper-parameter tuning by splitting the training set into folds, using time-series block evaluation. The folds are split by time interval and the time intervals do not overlap. The folds cover the full 2015-2018 timespan. For example, we created 5 folds in the 2015-2018 dataset. Each fold contained a "CV training dataset" of 200 days and "CV validation dataset" has the next 92 days after that. The idea of cross-validation is that we have multiple sets of train and validation sets so we do not overfit our model to one particular set of training data. Models that overfit the training set do not generalize well to unseen data. We calculate the cross-validation f-beta as the average f-beta score across all folds for each set of hyperparameters. Notably, for the baseline model measurement, we split the 3-month, Jan - March 2015 dataset into 3 folds where each training set was 20 days and the validation set was the subsequent 9 days.

We take a careful approach when performing data cleaning on each of the CV folds. Specifically, we do not want the broader information from the full dataset to leak into that particular CV folds. If that would happen, that risks information about the future being available incorrectly in the present during training. This data leakage would produce worse predictive models. To address this, we performed three data cleaning steps at the fold level, what we call Stage 2 Data Cleaning: (1) missing imputation, (2) standardization of numeric features and (3) creating our graph features. Specifically, we applied the meta-data from each CV training dataset to the corresponding validation dataset set within each fold.

In STEP 4, we estimated several models to get the Cross Validation metrics with all our features: random forest, XGBoost (LightGBM package), and neural networks. In the future, we plan to also try ensemble models. We adopted a bayesian hypertuning strategy appropriate for big data called Tree-structured Parazen Estimator (TPE) within the hyperopt package. TPE starts learning good values for your hyperparameters (within a range we set) as it goes through multiple trials. The bayesian approach is helpful for big data tuning because we do not have the compute resources to do a comprehensive grid search.

After we found our best hyperparameters for each estimator with cross-validation, in STEP 5, we prepared the full training dataset (2015-2018) and test set (2019) for our final model runs. Specifically, we complete the missing value imputation, normalization, and graph feature creation for both the training and test sets, appling the metadata from the 2015-2018 training dataset to the 2019 test set. Then, finally, in STEP 6, we do several more experiments on the full set of 2015-2018 training data to hone in our Final Model, using best model hyperparameters from STEP 4. The Final Model is the estimator that performs the best from STEP 6. In STEP 7, we use our Final Model to predict our flight delay outcome on the test set (2019). This gives us the final and best evaluation of our model, the f-beta score for the 2019 test set.

**Pipeline Image**
![Pipeline Image](https://i.imgur.com/wq62T0E.png)

## B. Families of Input Features
We divide our input features into families based on the specific data processing steps we must apply to them. The feature families used in our hypertuning and final models are below. We originally had 19 features, but once we one-hot encode the categorical features, we have **81 input columns**.
- **Numeric/Ratio (10 features)** are features that require a missing imputation strategy and normalization:  DISTANCE, ELEVATION, FE_PRIOR_DAILY_AVG_DEP_DELAY, FE_PRIOR_AVG_DURATION, FE_NUM_FLIGHT_SCHEDULED, DEP_DELAY_LAG, pagerank_origin, pagerank_dest, triangle_count_origin, triangle_count_origin
- **Categorical (9 features)** are features that we must one-hot encode, including a separate category for missing: DAY_OF_WEEK, MONTH, YEAR, OP_UNIQUE_CARRIER, CRS_DEP_BUCKET origin_type, dest_type, is_holiday, is_holiday_adjacent, IS_FIRST_FLIGHT_OF_DAY

## C. Experimental Modeling

In the Experimental Modeling section, we describe the series of experiments we performed on the training dataset to arrive at our final trained model. We describe our Baseline, our Hyperparameter Tuning, and our final runs on the full 2015-2018 training dataset. In our final runs, we also experimented with an innovative technique to downsample the training set during our final runs on the 2015-2018 dataset, which boosted performance. Please see the Data Dictionary above for a full list of features we used in our Baseline vs the Hypertuning and full 2015-2018 training set runs.

**Baseline.** We calculated our baseline with only 3 months of data from 2015 (with 3 cross-validation folds). Notably, the 3 months is a different time span than we used for hypertuning, so the baseline results are not apples-to-apples with the remaining cross-validation trials or final model runs. We considered two different estimators for our baseline model, a logistic regression and a random forest model, both with default parameters in the mLlib package. The baseline f-beta is a dismal 0.0 to 0.033, which is equivalent to the naive baseline that assumes all flights are on-time. A logistic model assumes that the target variable has a linear relationship with the feature variables, where a random forest model makes no such assumption. Logistic models are useful for its interpretability but more sophisticated models such as random forests and gradient-boosted trees tend to outperform the Logistic regression. Random forest is a bagging model that trains a lot of trees on samples of data (and also randomly samples features for each tree), aggregating the results. This approach can reduce the overfitting that may occur with a logistic regression. We expected the random forest to perform better than the logistic regression and also like that it does not assume a linear relationship between features and the outcome. We elected the random forest model as our baseline as we expect that the random forest model will far out-perform the logistic regression as we continue to refine our model and expand the time window of data.

|Estimator   |Dataset      |Key Parameters | Runtime | F-Beta (beta=2) Metric| 
|------------|-------------|---------------|----------------------|----------------------|
|Logistic Regression|Jan-Mar 2015 with 3 CV Folds|defaults: maxIter=100, regParam=0.0, elasticNetParam = 0.0| 1 min | 0.033                 |
|Random Forest (BASELINE)|Jan-Mar 2015 with 3 CV Folds|defaults: maxDepth=5, numTrees=20| 1 min |0                     |

**Hypertuning Experiments**. Hypertuning is the process of picking the parameters of your estimator that can optimize your f-beta score. For example, in tree-based estimators, a key parameter is the maximum depth of the tree. We did a series of experiments by hypertuning with the 2 most recent of our 5 Cross Validation folds that spanned part of 2017 and all of 2018 (200 days train, 92 days validation) on our estimators: random forest, LightGBM (a popular gradient boosted tree algorithm that is faster than XGBoost due to its leaf-wise (vertical) growth method, resulting in more loss reduction and higher accuracy on big data). While we originally intended to conduct the cross-validation on all 5 folds covering the full 2015-2018 period, we scaled down to 2 folds due to time-constraints. We used the hyperopt package with the Bayesian TPE algorithm. The bayesian algorithm learns the best parameters as it increases the number of trials. For example, we leveraged hyperopt’s capability to select from a uniform distribution with a defined minimum and maximum value. Since hyperopt “minimizes” your metric (f-beta), we minimized the negative f-beta score to get the appropriate result. We were constrained by time we only could try a limited number of hypertuning trials (8 random forest trials, 10 lightGBM trials, 2 neural network trials) and we focused on what we believe are the most impactful hyperparameters. The random forest trials and neural network runs took more time than the lightGBM trials. We believe we could do somewhat better if we had more time perform a more exhaustive hyperparameter search. Table 2 below shows the hyperparameter search space we used for each of our estimators and the Cross-Validation results.

- For the **random forest estimator**, we tuned the most important parameters, number of trees and maximum depth of the trees. We found the best f-beta score with 140 trees, with a maximum depth of each tree of 12 features.

- For **lightGBM** we tuned tree-specific parameters (e.g. maximum depth of the trees, minimum data in a leaf to control overfitting, subsample size), and regularization parameters (learning rate alpha) for xgboost, which can help reduce model complexity and enhance performance. We found the best f-beta score with max depth of 10 trees, minimum child weight of 2, minimum data in leaf of 1000, subsample of 0.8 and alpha rate 1. 
 
- For the **neural network**, we experimented with two different neural network architectures rather than a TPE hyperparameter search.  The neural network estimator is a classifier trainer based on the Multilayer Perceptron, with sigmoid activation functions at each layer. The input layer has 1 node per feature and the output layer has softmax with 2 nodes (i.e., binary classification). The first architecture layer specification is 81 (Input Layer) - 32 (Hidden Layer w/ Sigmoid Activation) - 2 (Output Layer w/ Softmax Activation). The second layer specification is 81 (Input Layer) - 8 (Hidden Layer #1 w/ Sigmoid Activation) – 4 –  (Hidden Layer #2 w/ Sigmoid Activation) - 2 (Output Layer w/ Softmax Activation). We found similar results for both models, but the latter performed slightly better.

 In summary, the neural network performed the best with a f-beta score of 43% followed by random forest with a f-beta of 40% (See Table 1). The lightGBM performed the worst, with a f-beta of 29%, so we dropped this estimator from the final model runs with the full 2015-2018 training data.

**Table 2. Hypertuning Experiments (3 Hypertuning Experiments using Cross-Validation Folds)**
|#           |Estimator    |Dataset        |Best Hyperparameters|Runtime of HyperTuning|Number of evaluation trials                                                |Best F-Beta (beta=2) Metric|
|------------|-------------|---------------|--------------------|----------------------|---------------------------------------------------------------------------|---------------------------|
|1           |Random Forest|2017-2018 (2-folds). train: 200 days, val: 92|maxDepth=5, numTrees=20 <br/><br/>**Search Space**: "maxDepth": hp.quniform("maxDepth", 4, 12, 2), "numTrees": hp.quniform("numTrees", 20, 200, 20)|8 hours               |8                                                                          |40.1%                     |
|2           |LightGBM     |2017-2018 (2-folds). train: 200 days, val: 92|maxDepth=10 Min_child_weight=2 Min_data_in_leaf=1000 Subsample=0.8 reg_alpha=1maxDepth=10, min_child_weight=2, min_data_in_leaf=1000, subsample=0.8, reg_alpha=1 <br/><br> **Search Space**: "maxDepth": hp.quniform("maxDepth", 2, 10, 1), 'max_depth':hp.quniform("max_depth", 3,10,2), ‘min_child_weight':hp.quniform("min_child_weight", 1,6,2), 'min_data_in_leaf':hp.quniform("min_data_in_leaf", 100, 1000, 200), 'subsample':hp.choice("subsample", [i/10.0 for i in range(6,10)]), 'reg_alpha':hp.choice("reg_alpha", [1e-5, 1e-2, 0.1, 1, 100])|3 hours               |10                                                                         |28.7%                     |
|3           |Neural Network|2017-2018 (2-folds). train: 200 days, val: 92|layers=[81, 8, 4, 2], maxIter=100, blockSize=128, stepSize=0.03|3 hours               |2                                                                          |**43.4%**                 |


**Final Runs on Full 2015-2018 Training Set.** We ran the next set of experiments on the full 2015-2018 training dataset. We present both the f-beta and the AUC Metric for greater context (AUC = Area Under the Curve, where the x-axis is the false positive rate and the y-axis the true positive rate). The AUC metric is calculated using the predicted probabilities of each flight and represents how well the model can distinguish between outcome classes, ranging from 50% to 100%, with 50% as good as random chance and 100% as a perfect score. For experiments, 1-2 in Table 3, we applied the best hyperparameters from the random forest and neural network estimators to train our full 2015-2018 training set. In experiments 4 and 5, we applied an innovative, statistical downsampling technique to the training set, again on both a random forest and neural network model. Downsampling allows us to create a balanced training set with an equal number of delayed and on-time flights (see Table 3). We retain all delayed flights and randomly down-sampled the on-time ones to achieve this balance. As a technical note related to data leakage, it's important to note that this down-sampling is limited to the training data only, not affecting the test dataset. The approach aims to ensure that our models learn to discriminate between classes using an equal number of examples, leading to better generalization. When deployed in production, the model will make predictions on a more significant number of on-time flights than delayed ones, but its training on a balanced dataset enhances its overall performance. The distribution of the downsampled dataset is shown in Table 4. 

Experiment 5, a neural network with 2 hidden layers trained on a downsampled 2015-2018 training set, has the highest f-beta score, with a f-beta score of 63.1%. The corresponding AUC is also one of the highest, at 71.8%.

**Table 3: Experiments on 2015-2018 Training Dataset (5 Experiments)**
|#|Estimator   |Dataset      |Hyperparameters|Runtime|F-Beta (beta=2) Metric|AUC Metric|
|-|------------|-------------|---------------|----------------------|----------|-------|
|1|Neural Network 1|2015-2018 Train|layers=[81, 8, 4, 2], maxIter=100, blockSize=128, stepSize=0.03| 20 min| 40.5%                |66.9%    |
|2|Neural Network 2|2015-2018 Train|layers=[81, 32, 16, 4, 2], maxIter=100, blockSize=128, stepSize=0.03| 50 min |40.3%                |67.0    |
|3|Random Forest 1|2015-2018 Train|maxDepth=5, numTrees=20| 50 min | 40.5%                |50.3%    |
|4|Random Forest 2 |2015-2018 Downsampled Training Set|maxDepth=5, numTrees=20| 35 min |60.4%        |72.0|
|5|Neural Network 3|2015-2018 Downsampled Training Set|layers=[81, 8, 4, 2], maxIter=100, blockSize=128, stepSize=0.03| 21 min | **63.1%**     |71.8%|

**Table 4. Distribution on the Outcome Variable "DEP_DEL15" Before and After Downsampling (Canceled Flights Removed)**
|Label    |Training Dataset Row Count (original) <br/> 2015-2018|Training Dataset After Downsampling Row Count <br/> 2015-2018|Test Dataset Row Count<br/> 2019|
|---------|-----------------------------------------------------|-------------------------------------------------------------|--------------------------------|
|No Delay |19,607,943                                           |4,312,859                                                    |5,905,453                       |
|Delay    |4,312,859                                            |4,312,859                                                    |1,353,702                       |
|**Total**    |**23,920,802**                                           |**8,625,718**                                                    |**7,259,155**                       |


**Loss Functions for Estimators.** For completeness, we provide the loss functions for our estimators we considered are below.
- **Logistic Regression, Gradient Descent, and Neural Network Backpropagation**: The logistic loss function is used for our standard logistic regression model as well as for the optimization of the weights and biases during back-prop learning.

$$ \text{LogisticLoss} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \cdot \log(p_i) + (1 - y_i) \cdot \log(1 - p_i) \right] $$

Where:
$$ N\text{: the number of flights} $$
$$ y_i\text{: the true label (1 if flight was actually delayed)} $$
$$ p_i\text{: the predicted probability that the i-th flight would be delayed} $$


- **Random Forest and XGBoost**: For our Random Forest and XGBoost models, we employ the entropy-based information gain loss function with each split in each tree.
$$ H(X) = I_E(p_1, p_2, ..., p_J) = - (p_1 \cdot \log_2(p_1) + p_2 \cdot \log_2(p_2)) $$

Where:
$$ J\text{: the number of classes (2 for binary classification)} $$
$$ p_i\text{: the proportion of samples belonging to class i in a particular node} $$

# VI. Results

Overall, we see that we have made marked improvement from our baseline (f-beta = 0.0) using 3 months of training data to our hyperparameter tuning with cross-fold validation on training data from 2017 and 2018 (f-beta=43.4%) to our final model runs on the full downsampled 2015-2018 training data (f-beta = 63.1%). The biggest takeaway during our 5 experiments in Table 2 is the power of downsampling the majority class in the training set to address the challenges associated with predicting a rare outcome.

### A. Heldout 2019 Test Set
**2019 Holdout Results.** Using the best model from Table 3 (Experiment #5), we evaluated our holdout unseen test set with the full year of 2019 flight data. Our final f-beta metric on the unseen 2019 test dataset is 54.4%, as shown in Table 5 below. This performance drop from the training set to unseen data is expected. Our model is fit to the training data from 2015-2018 and therefore will not generalize as well to an unseen holdout dataset from 2019. Next, we present a gap analysis of the predictions on the 2019 Holdout Dataset to highlight where we are predicting well and where we are predicting less well, helping to guide future feature engineering work.

**Table 5: 2019 Holdout Data Result**
|Estimator   |Dataset      |Hyperparameters|F-Beta (beta=2) Metric|AUC Metric|
|------------|-------------|---------------|----------------------|----------|
|Neural Network 1|2019 Holdout|layers=[81, 8, 4, 2], maxIter=100, blockSize=128, stepSize=0.03| 54.4% | 71.0  |

### B. Gap Analysis: Analysis of False Negatives

In this section, we conduct a post-mortem gap analysis on the prediciton on the 2019 heldout, test set to examine patterns in our prediction errors to gain insight into where this model could be improved. To examine the gap between the model's expected performance and the actual results, we will evaluate how the f-beta score differs across the different airlines as well as over different times throughout the week. Here we just examine the holdout test data covering 2019, because we are most interested in the error patterns that arise with data the model has not seen before.

**Model Performance by Airline**

First we look at how consistently our model performs across all airlines, big and small. Do we see a pattern that could indicate a systemic issue we might flag for future work?

![Model Performance by Airline](https://i.imgur.com/Btr1TIq.png)

**Insights**
* From this plot, we can see that our model handles small airports well, except for Hawaii Air (HA) and Alaska Air (AS) who perform many more long-haul flights considering that they are headquartered outside of the continental United States. 
* Surprisingly, our model significantly underperforms on the major airlines United (UA), American (AA), and Delta (DL). This observation suggests the need for a closer examination of the underlying factors affecting the prediction accuracy for these airlines.
* Southwest Airlines (WN) emerges as a carrier that our model predicts with relative ease, potentially due to distinctive operational characteristics or patterns that align well with our model's features.

**Model Performance by Time of Week**

Let's delve into the consistency of the model's performance across the different times of the week. In the below chart, we round down the departure time to the hour. 

![Heatmap of F2 Scores by Day of Week and Hour](https://i.imgur.com/JFcIrox.png)

**Insights**
* The model's performance is notably weak during times when flights are infrequent, particularly flights scheduled before 8:00 a.m.
* As the day progresses and more flights take place, the model's performance improves significantly.
* The model demonstrates better performance on weekdays compared to weekends.
* The fact that the model performs better where there are more consecutively booked flights is promising - those are the flights airlines struggle the most with keeping on-time due to congestion.

**Gap Analysis Recommendations for Further Improvement**

Based on the above findings, we recommend the following feature engineering work done to further improve the model: 

* **Cross-Oceanic Flight Indicator**: Include a feature that distinguishes airlines that primarily carry out cross-oceanic flights. Currently, we account for the distance of the flight but there could also be a factor when traveling outside the contiguous 48 US states. 
* **Hour-of-Day Segmentation**: Include a feature that breaks out hour of the day similar to quartiles with differently-lengthed time spans. For example, there may be as many flights departing between 12am and 10am as there are departing between 10am and noon or between 5pm and 6pm. 
* **Weekend Indicator**: Provide a binary variable distinguishing the weekend from the weekdays to directly capture the difference in demands. Perhaps this difference stems from business vs. leisure travel.

# VII. Conclusions and Next Steps

Our goal was to create a machine learning classifier that reliably predicts whether a flight will be delayed 2 hours prior to its departure. This tool will help the airline industry improve their daily operations and increase their profit. We focused on a f-beta metric that errors on the side of predicting more flight delays and being wrong, with a goal of achieving at least the 80% benchmark. The predictive power of our model, with a f-beta score of 54.4% on the 2019 holdout test set, is quite low, and so we will continue to work to refine this tool. Our best estimator was a neural network with two hidden layers and the random forest estimator came in at a close second. We believe foremost we should improve the set of features, and secondly, continue to look for better estimators, including ensemble models, and hyperparameters to make smaller improvements. We will also look to feature selection methods such as Shap values to further refine our feature set. We found great power in random downsampling the training set to fit our model and can also continue to try other non-random downsampling approaches such as downsampling at the day- or week-level. We built a robust data pipeline that can rapidly integrate new features into the model and minimize data leakage.

Our gap analysis showed that we predicted delays for some airline carriers better than others and we also seen variation by time of day and day of the week. For certain, we are not fully capturing the myriad of factors that contribute to a flight's delay. We need more predictive features that can capture daily operational challenges of the airport environment that could lead to delays such as staff scheduling issues and airplane maintenance schedules. We plan to improve this model by doing more feature engineering and conducting hyperparameter tuning over more trials. There are many opportunities for more features, such as those identified in our gap analysis, creating daily or weekly seasonal trend features, an inclement weather feature, and interaction terms between month and holiday (MONTH x HOLIDAY) or OP_UNIQUE_CARRIER*DAY_OF_WEEK. We also might pull in more datasets that represent the more event-based features like the Superbowl, airport maintenance, and airline reputation.

**Scalability concerns/Challenges/Learnings when working with Big Time-Series Data**. This project was an excellent introduction to machine learning approaches with time-series, big data. We learned that we must primarily work with a small subset of the data for hyperparameter tuning as tuning a 4 year training set with cross-validation folds can easily take most of a day to run. However, doing some of the training experiments on a different subset of data than you use for your final model runs presents some new challenges as well. We were unsure how well our cross-validation results on our subset will generalize to the broader training set (although in our case the results were the same across both datasets). The size of the data also affects the complexity of the model we would attempt. For example, we attempted only a handful of modest neural network architectures. Simillary, we performed only a limited amount of hyperparameter tuning due to the compute constraints of big data within the short deadline of this project. Our bayesian hyper tuning approach that learns how to pick better parameters is an exciting approach to optimize our model. We also faced productivity constraints as a team due to the cluster size constraints in the cloud. This was a group project and we also had to sequence our work so that only one team member was working on the cloud data cluster at a time once we were working with the full 5-year dataset. Unfortunately, due to additional processing time needed to complete the main model runs, we were not able to implement the secondary Prophet model that we intended to ensemble. But it was tested on smaller data sets and trained successfully on the full final dataset.

**Innovations**. We offered several innovative elements to our approach, namely trying a statistical downsampling procedure to address class imbalance that dramatically improved our model's performance. We also created sophisticated graph features (i.e., pagerank and triangle count) to represent the role of each airport in the network and tried using a special LightGBM package that provides speedy results for gradient boosted trees.